In [1]:
# Set all the parameters for the training

Name = 'My Instrument' #@param {type:"string"}
Name = Name.replace(' ', '_')

Training_Steps = 30000

Ignore_Previous = False

Google_Drive = False

In [2]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import glob
import os
import shutil
import time
import IPython
import json
import subprocess

import tensorflow as tf

from ipyfilechooser import FileChooser

2024-06-04 14:26:51.452613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def get_audio_files(drive_dir, audio_dir):
    if drive_dir:
        # List all files in the drive_dir
        all_files = os.listdir(drive_dir)
        
        # Filter out files that are not .mp3 or .wav
        audio_paths = [f for f in all_files if f.endswith('.mp3') or f.endswith('.wav')]
        audio_paths = [os.path.join(drive_dir, f) for f in audio_paths]  # Full paths
        
        if len(audio_paths) < 1:
            raise FileNotFoundError("Sorry, it seems that there aren't any MP3 or "
                                    f"WAV files in your folder ({drive_dir}). Try "
                                    "running again and choose a different folder.")
    else:
        # If no drive_dir is specified, handle as per your environment.
        # This part might need to be modified based on where you're uploading files from.
        print("Please specify a directory containing audio files.")
        return
    
    # Copy Audio.
    print('Copying audio to local directory for training...')
    for src in audio_paths:
        target = os.path.join(audio_dir, 
                              os.path.basename(src).replace(' ', '_'))
        print('Copying {} to {}'.format(src, target))
        shutil.copy(src, target)

In [4]:
# test get_audio_files
get_audio_files("samples/","audio/")

Copying audio to local directory for training...
Copying samples/hls.mp3 to audio/hls.mp3


In [5]:
import os

def directory_has_files(target_dir):
    n_files = len([f for f in os.listdir(target_dir) if os.path.isfile(os.path.join(target_dir, f))])
    return n_files > 0



def prepare_dataset(audio_dir, 
                    data_dir,
                    sample_rate=16000, 
                    frame_rate=50, 
                    example_secs=4.0, 
                    hop_secs=1.0, 
                    viterbi=True, 
                    center=True):
  if directory_has_files(data_dir):
    print(f'Dataset already exists in `{data_dir}`')
    return
  else:
    # Otherwise prepare new dataset locally.
    print(f'Preparing new dataset from `{audio_dir}`')

    print()
    print('Creating dataset...')
    print('This usually takes around 2-3 minutes for each minute of audio')
    print('(10 minutes of training audio -> 20-30 minutes)')

    audio_filepattern = os.path.join(audio_dir, '*')
    audio_fp_str = f'"{audio_filepattern}"'    
    tfrecord_path_str = f'"{data_dir}/train.tfrecord"'

    # print what we are going to run
    print("Running the following command:")
    print(f"!ddsp_prepare_tfrecord --input_audio_filepatterns={audio_fp_str} --output_tfrecord_path={tfrecord_path_str} --num_shards=10 --sample_rate={sample_rate} --frame_rate={frame_rate} --example_secs={example_secs} --hop_secs={hop_secs} --viterbi={viterbi} --center={center}")

    !ddsp_prepare_tfrecord \
    --input_audio_filepatterns=$audio_fp_str \
    --output_tfrecord_path=$tfrecord_path_str \
    --num_shards=10 \
    --sample_rate=$sample_rate \
    --frame_rate=$frame_rate \
    --example_secs=$example_secs \
    --hop_secs=$hop_secs \
    --viterbi=$viterbi \
    --center=$center &> /dev/null

In [6]:
# test get_audio_files

def get_model_dir(base_dir, ignore_previous=False):
    base_str = 'ddsp-training'
    # List all items in base_dir
    all_items = os.listdir(base_dir)
    
    # Filter out directories that match the pattern
    dirs = [item for item in all_items if os.path.isdir(os.path.join(base_dir, item)) and item.startswith(base_str)]
    
    # Sort the directories to find the most recent
    dirs.sort()
    
    if dirs and not ignore_previous:
        model_dir = os.path.join(base_dir, dirs[-1])  # Use the last directory as the most recent
    else:
        # If no suitable directory is found or if ignoring previous directories, create a new one.
        now = datetime.datetime.now().strftime('%Y-%m-%d-%H%M')
        model_dir = os.path.join(base_dir, f'{base_str}-{now}')
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)  # Create the directory since it does not exist
    return model_dir


def reset_state(data_dir, audio_dir, model_dir):
    if os.path.exists(data_dir):
        shutil.rmtree(data_dir)
        shutil.rmtree(audio_dir)
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(audio_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)


def export_and_download(model_dir, model_name=Name):
  export_path = os.path.join(model_dir, model_name)

  model_dir_str=f'"{model_dir}"'
  export_path_str=f'"{export_path}"'
  
  !ddsp_export \
  --name=$model_name \
  --model_path=$model_dir_str \
  --save_dir=$export_path_str \
  --inference_model=vst_stateless_predict_controls \
  --tflite \
  --notfjs

  # Zip the whole directory.
  zip_fname = f'{model_name}.zip'
  zip_fp = os.path.join(model_dir, zip_fname)
  print(f'Export complete! Zipping {export_path} to {zip_fp}')
  print(f'Zipping Complete! Downloading... {zip_fname}')
  print(f'You can also find your model at {export_path}')

def train(model_dir, data_dir, steps=30000):
  file_pattern = os.path.join(data_dir, 'train.tfrecord*')
  fp_str = f"TFRecordProvider.file_pattern='{file_pattern}'"
  !ddsp_run \
  --mode=train \
  --save_dir="$model_dir" \
  --gin_file=models/vst/vst.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="$fp_str" \
  --gin_param="TFRecordProvider.centered=True" \
  --gin_param="TFRecordProvider.frame_rate=50" \
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=$steps" \
  --gin_param="train_util.train.steps_per_save=300" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=3"

def get_gpu_type():
    gpu_info = []
    try:
        bash_command = "nvidia-smi --query-gpu=name --format=csv"
        output = subprocess.getoutput(bash_command)
        lines = output.split("\n")
        lines.pop(0)
        return lines[0]
    except OSError:
        print("GPU device is not available")
        return ''


# test get_audio_files

def get_model_dir(base_dir, ignore_previous=False):
    base_str = 'ddsp-training'
    # List all items in base_dir
    all_items = os.listdir(base_dir)
    
    # Filter out directories that match the pattern
    dirs = [item for item in all_items if os.path.isdir(os.path.join(base_dir, item)) and item.startswith(base_str)]
    
    # Sort the directories to find the most recent
    dirs.sort()
    
    if dirs and not ignore_previous:
        model_dir = os.path.join(base_dir, dirs[-1])  # Use the last directory as the most recent
    else:
        # If no suitable directory is found or if ignoring previous directories, create a new one.
        now = datetime.datetime.now().strftime('%Y-%m-%d-%H%M')
        model_dir = os.path.join(base_dir, f'{base_str}-{now}')
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)  # Create the directory since it does not exist
    return model_dir


def reset_state(data_dir, audio_dir, model_dir):
    if os.path.exists(data_dir):
        shutil.rmtree(data_dir)
        shutil.rmtree(audio_dir)
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(audio_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)


def export_and_download(model_dir, model_name=Name):
  export_path = os.path.join(model_dir, model_name)

  model_dir_str=f'"{model_dir}"'
  export_path_str=f'"{export_path}"'
  
  !ddsp_export \
  --name=$model_name \
  --model_path=$model_dir_str \
  --save_dir=$export_path_str \
  --inference_model=vst_stateless_predict_controls \
  --tflite \
  --notfjs

  # Zip the whole directory.
  zip_fname = f'{model_name}.zip'
  zip_fp = os.path.join(model_dir, zip_fname)
  print(f'Export complete! Zipping {export_path} to {zip_fp}')
  print(f'Zipping Complete! Downloading... {zip_fname}')
  print(f'You can also find your model at {export_path}')

def train(model_dir, data_dir, steps=30000):
  file_pattern = os.path.join(data_dir, 'train.tfrecord*')
  print(file_pattern)
  fp_str = f"TFRecordProvider.file_pattern='{file_pattern}'"
  print(fp_str)
  !ddsp_run \
  --mode=train \
  --save_dir="$model_dir" \
  --gin_file=models/vst/vst.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="$fp_str" \
  --gin_param="TFRecordProvider.centered=True" \
  --gin_param="TFRecordProvider.frame_rate=50" \
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=$steps" \
  --gin_param="train_util.train.steps_per_save=300" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=3"

def get_gpu_type():
    gpu_info = []
    try:
        bash_command = "nvidia-smi --query-gpu=name --format=csv"
        output = subprocess.getoutput(bash_command)
        lines = output.split("\n")
        lines.pop(0)
        return lines[0]
    except OSError:
        print("GPU device is not available")
        return ''

def log_event(event_name, event_details):
  """Log event with name and details dictionary."""
  details_json = json.dumps(event_details)
  js_string = "gtag('event', '%s', %s);" % (event_name, details_json)
  IPython.display.display(IPython.display.Javascript(js_string))

In [7]:
def run_training(drive_dir=''):
  log_event('runTrainingStarted', {})
  # ------------------------------------------------------------------------------
  # Install DDSP here to allow selecting folder first
  # ------------------------------------------------------------------------------
  print('Installing DDSP...')
  print('This should take about 2 minutes...')

  # ------------------------------------------------------------------------------
  # Import DDSP
  # ------------------------------------------------------------------------------
  from ddsp.colab import colab_utils
  globals()['colab_utils'] = colab_utils

  # ------------------------------------------------------------------------------
  # Setup
  # ------------------------------------------------------------------------------
  # Save data locally, but model on drive.
  data_dir = 'data/'
  audio_dir = 'audio/'
  model_dir = get_model_dir(drive_dir)

  reset_state(data_dir, audio_dir, model_dir)

  # ------------------------------------------------------------------------------
  # Dataset
  # ------------------------------------------------------------------------------
  tick = time.time()

  get_audio_files(drive_dir, audio_dir)
  prepare_dataset(audio_dir, data_dir)

  log_event('datasetMins', {'value': round((time.time() - tick) // 60)})


  # ------------------------------------------------------------------------------
  # Train
  # ------------------------------------------------------------------------------
  tick = time.time()

  print()
  print('Training...')
  train(model_dir, data_dir, steps=Training_Steps)

  log_event('trainMins', {
      'event_category': str(Training_Steps),
      'value': round((time.time() - tick) // 60),
  })


  # ------------------------------------------------------------------------------
  # Export
  # ------------------------------------------------------------------------------
  tick = time.time()

  print()
  print('Exporting model...')
  export_and_download(model_dir)

  log_event('exportMins', {'value': round((time.time() - tick) // 60)})

In [8]:
get_gpu_type()

'NVIDIA GeForce RTX 2070'

In [9]:
def run_training(drive_dir=''):
  log_event('runTrainingStarted', {})
  # ------------------------------------------------------------------------------
  # Install DDSP here to allow selecting folder first
  # ------------------------------------------------------------------------------
  print('Installing DDSP...')
  print('This should take about 2 minutes...')


  # ------------------------------------------------------------------------------
  # Setup
  # ------------------------------------------------------------------------------
  # Save data locally, but model on drive.
  data_dir = 'data/'
  audio_dir = 'audio/'
  model_dir = get_model_dir(drive_dir)

  reset_state(data_dir, audio_dir, model_dir)

  # ------------------------------------------------------------------------------
  # Dataset
  # ------------------------------------------------------------------------------
  tick = time.time()

  get_audio_files(drive_dir, audio_dir)
  prepare_dataset(audio_dir, data_dir)

  log_event('datasetMins', {'value': round((time.time() - tick) // 60)})


  # ------------------------------------------------------------------------------
  # Train
  # ------------------------------------------------------------------------------
  tick = time.time()

  print()
  print('Training...')
  train(model_dir, data_dir, steps=Training_Steps)

  log_event('trainMins', {
      'event_category': str(Training_Steps),
      'value': round((time.time() - tick) // 60),
  })


  # ------------------------------------------------------------------------------
  # Export
  # ------------------------------------------------------------------------------
  tick = time.time()

  print()
  print('Exporting model...')
  export_and_download(model_dir)

  log_event('exportMins', {'value': round((time.time() - tick) // 60)})


In [10]:
def run():
  """Create and display a FileChooser widget."""
  log_event('runStarted', {})
  gpu_type = get_gpu_type()
  print(f'Using a {gpu_type} GPU...')
  log_event('gpuType', {'event_category': gpu_type})

  print('Upload Audio Manually...')
  run_training(drive_dir='./samples')


In [11]:
run()

<IPython.core.display.Javascript object>

Using a NVIDIA GeForce RTX 2070 GPU...


<IPython.core.display.Javascript object>

Upload Audio Manually...


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>


Training...
data/train.tfrecord*
TFRecordProvider.file_pattern='data/train.tfrecord*'
2024-06-04 14:26:54.145890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
/usr/local/lib/python3.8/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
2024-06-04 14:26:57.084461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node z

<IPython.core.display.Javascript object>


Exporting model...
2024-06-04 14:26:58.945484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
/usr/local/lib/python3.8/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
2024-06-04 14:27:02.588481: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-04 14:27:02.592408: I tensorflow/compiler/xla/stream_ex

<IPython.core.display.Javascript object>

In [12]:
!pip install ffmpeg-downloader


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [13]:
!ffdl install --add-path -y

In [15]:
!ddsp_prepare_tfrecord --input_audio_filepatterns="audio/*" --output_tfrecord_path="data//train.tfrecord" --num_shards=10 --sample_rate=16000 --frame_rate=50 --example_secs=4.0 --hop_secs=1.0 --viterbi=True --center=True

2024-06-04 14:27:57.019437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
/usr/local/lib/python3.8/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
2024-06-04 14:28:00.075152: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-04 14:28:00.077395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu